In [95]:
#Libraries
import pypyodbc
import pandas as pd
#Conn and credentials
conn_str = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:servidortcprueba.database.windows.net,1433;Database=db_campechanos;Uid=adminraul;Pwd=5ccu9$j$XzhNMMF;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
connection = pypyodbc.connect(conn_str)
cursor = connection.cursor()

#Training dataset
query = "SELECT * FROM train"
cursor.execute(query)
rows = cursor.fetchall()
df_train = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_train.columns = columns

#Test dataset, dont have sales price
query = "SELECT * FROM test"
cursor.execute(query)
rows = cursor.fetchall()
df_test = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_test.columns = columns

#Sample dataset, id and sales price
query = "SELECT * FROM sample"
cursor.execute(query)
rows = cursor.fetchall()
df_sample = pd.DataFrame(rows)

columns = [column[0] for column in cursor.description]
df_sample.columns = columns


In [115]:
num_nulos_por_columna = df_train.isnull().sum()
variable_dependiente = 'saleprice'

correlations = df_train.corr()[variable_dependiente]

abs_correlations = correlations.abs()
correlations_df = pd.DataFrame({ 'Correlación': correlations})

correlations_df = correlations_df.sort_values(by='Correlación', ascending=False)

correlations_df['Nulos'] = num_nulos_por_columna
list_to_save=[]
for index, row in correlations_df.iterrows():
    if abs(row['Correlación']) >= 0.6:
        list_to_save.append(row.name)
df_train = df_train[list_to_save]

In [116]:
train = df_train
train.fillna(train.mean(), inplace=True)

In [117]:
num_nulos_por_columna = df_train.isnull().sum()
variable_dependiente = 'saleprice'

correlations = df_train.corr()[variable_dependiente]

abs_correlations = correlations.abs()
correlations_df = pd.DataFrame({ 'Correlación': correlations})

correlations_df = correlations_df.sort_values(by='Correlación', ascending=False)

correlations_df['Nulos'] = num_nulos_por_columna

print(correlations_df)

             Correlación  Nulos
saleprice       1.000000      0
overallqual     0.790982      0
grlivarea       0.708624      0
garagecars      0.640409      0
garagearea      0.623431      0
totalbsmtsf     0.613581      0
_1stflrsf       0.605852      0
kitchenqual    -0.659600      0
exterqual      -0.682639      0


In [118]:
# Dividir el conjunto de datos en características (X) y objetivo (y)
X = train.drop('saleprice', axis=1)  # Características
y = train['saleprice']  # Objetivo

In [119]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba (por ejemplo, 80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [120]:
from sklearn.linear_model import LinearRegression

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo en el conjunto de entrenamiento
model.fit(X_train, y_train)

LinearRegression()

In [121]:
from sklearn.metrics import mean_squared_error

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

Error Cuadrático Medio (MSE): 1438769013.9527407


In [122]:
# Importa las bibliotecas necesarias
from sklearn.ensemble import RandomForestRegressor  # Por ejemplo, puedes probar Random Forest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

# Ingeniería de Características (si es necesario)
# Puedes crear nuevas características o transformar las existentes aquí

# Crea y ajusta un modelo de Random Forest (o el modelo que elijas)
model = RandomForestRegressor(n_estimators=100, random_state=0)  # Ajusta los hiperparámetros según sea necesario
model.fit(X_train, y_train)

# Realiza la validación cruzada para obtener una estimación del rendimiento
mse_scores = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(mse_scores)

# Calcula el promedio de RMSE (Root Mean Squared Error)
avg_rmse = np.mean(rmse_scores)
print("Promedio de RMSE:", avg_rmse)

# Ahora puedes usar el modelo entrenado para hacer predicciones en tus datos de prueba (test), si los tienes

# Si deseas ajustar más el modelo, puedes considerar la optimización de hiperparámetros u otras técnicas de mejora


Promedio de RMSE: 34384.94467609102


In [124]:
# Verificar si hay columnas faltantes en df_test en comparación con df_train
missing_columns = set(df_train.columns) - set(df_test.columns)

# Agregar las columnas faltantes en df_test y completarlas con valores nulos
for column in missing_columns:
    df_test[column] = None  # Puedes cambiar None por el valor que desees para los nulos


In [125]:
df_cols =df_train.drop('saleprice', axis=1)
column_order = df_cols.columns

# Reorganizar las columnas en df_test de acuerdo al orden en df_train
df_test = df_test[column_order]
df_test.fillna(df_test.mean(), inplace=True)
df_test['is_house_25fstory'] = 0

In [127]:

X_test = df_test  # Asume que el conjunto de prueba tiene las mismas características que el conjunto de entrenamiento
predictions = model.predict(X_test)

# Crea un DataFrame con las predicciones
predictions_df = pd.DataFrame({'SalePrice': predictions})

# Guarda las predicciones en un archivo CSV
predictions_df.insert(0, 'id', range(1461, len(predictions_df) + 1461))
predictions_df.to_csv('predictions.csv', index=False)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- is_house_25fstory
